In [52]:
import numpy as np
import pandas as pd
import spacy
import string
import pickle as pkl
from sklearn import preprocessing
import os.path
from operator import add

In [2]:
#load in the data
train_df = pd.read_csv('snli_train.tsv', sep="\t")
val_df = pd.read_csv('snli_val.tsv',sep="\t")

#get data & convert sentences to lists
train_sentence1=train_df['sentence1'].values.tolist()
train_sentence2=train_df['sentence2'].values.tolist()
val_sentence1=val_df['sentence1'].values.tolist()
val_sentence2=val_df['sentence2'].values.tolist()

#convert the text labels to numeric
le = preprocessing.LabelEncoder()
le.fit(train_df['label'])
train_targets=le.transform(train_df['label']).tolist()
val_targets=le.transform(val_df['label']).tolist()

In [3]:
# Load English tokenizer, tagger, parser, NER and word vectors
tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation

# lowercase and remove punctuation
def tokenize(sent):
    tokens = tokenizer(sent)
    return [token.text.lower() for token in tokens if (token.text not in punctuations)]

In [4]:
# This is the code cell that tokenizes train/val/test datasets
def tokenize_dataset(dataset):
    token_dataset = []
    # we are keeping track of all tokens in dataset 
    # in order to create vocabulary later
    all_tokens = []
    
    for sample in dataset:
        tokens = tokenize(sample)
        token_dataset.append(tokens)
        all_tokens += tokens

    return token_dataset, all_tokens

In [5]:
#train set tokens
if not os.path.exists('train_all_combined_sentence_tokens.p'):
    print("Tokenizing train data")
    train_sentence1_tokens, train_all_sentence1_tokens = tokenize_dataset(train_sentence1)
    train_sentence2_tokens, train_all_sentence2_tokens = tokenize_dataset(train_sentence2)
    train_all_combined_sentence_tokens = train_all_sentence1_tokens + train_all_sentence2_tokens
    pkl.dump(train_all_combined_sentence_tokens, open("train_all_combined_sentence_tokens.p", "wb"))

In [6]:
#if pickle files do not exist, collect them.  Otherwise, this if block will not run
if not os.path.exists('train_sentence1_tokens.p'):
    print ("Tokenizing train data")
    train_sentence1_tokens, train_all_sentence1_tokens = tokenize_dataset(train_sentence1)
    train_sentence2_tokens, train_all_sentence2_tokens = tokenize_dataset(train_sentence2)
    pkl.dump(train_sentence1_tokens, open("train_sentence1_tokens.p", "wb"))
    pkl.dump(train_sentence2_tokens, open("train_sentence2_tokens.p", "wb"))
    #pkl.dump(train_all_sentence1_tokens, open("train_all_sentence1_tokens.p", "wb"))
    #pkl.dump(train_all_sentence2_tokens, open("train_all_sentence2_tokens.p", "wb"))1

    #combine tokens from both sentences to create a shared dictionary
    train_all_combined_sentence_tokens = train_all_sentence1_tokens + train_all_sentence2_tokens
    pkl.dump(train_all_combined_sentence_tokens, open("train_all_combined_sentence_tokens.p", "wb"))

    #val set tokens
    print ("Tokenizing val data")
    val_sentence1_tokens, _ = tokenize_dataset(val_sentence1)
    val_sentence2_tokens, _ = tokenize_dataset(val_sentence2)
    pkl.dump(val_sentence1_tokens, open("val_sentence1_tokens.p", "wb"))
    pkl.dump(val_sentence2_tokens, open("val_sentence2_tokens.p", "wb"))

In [7]:
#If you have previously run the previous cell, run this cell instead to load preprocessed datasets
train_sentence1_tokens = pkl.load(open("train_sentence1_tokens.p", "rb"))
train_sentence2_tokens = pkl.load(open("train_sentence2_tokens.p", "rb"))
train_all_combined_sentence_tokens = pkl.load(open("train_all_combined_sentence_tokens.p", "rb"))
train_all_sentence1_tokens = pkl.load(open("train_all_sentence1_tokens.p", "rb"))
train_all_sentence2_tokens = pkl.load(open("train_all_sentence2_tokens.p", "rb"))
val_sentence1_tokens = pkl.load(open("val_sentence1_tokens.p", "rb"))
val_sentence2_tokens = pkl.load(open("val_sentence2_tokens.p", "rb"))

In [8]:
#print information about the token datasets
# double checking
print ("Train sentence1 dataset size is {}".format(len(train_sentence1_tokens)))
print ("Train sentence2 dataset size is {}".format(len(train_sentence2_tokens)))
print ("Val sentence1 dataset size is {}".format(len(val_sentence1_tokens)))
print ("Val sentence2 dataset size is {}".format(len(val_sentence2_tokens)))

print ("\nTotal number of tokens in sentence1 train dataset is {}".format(len(train_all_sentence1_tokens)))
print ("Total number of tokens in sentence2 train dataset is {}".format(len(train_all_sentence2_tokens)))
print ("Total number of tokens in combined sent1 & sent2 train dataset is {}".format(len(train_all_combined_sentence_tokens)))

print ("\nTotal number of *unique* tokens in sentence1 train dataset is {}".format(len(set(train_all_sentence1_tokens))))
print ("Total number of *unique* tokens in sentence2 train dataset is {}".format(len(set(train_all_sentence2_tokens))))
print ("Total number of *unique* tokens in sent1 & sent2 train dataset is {}".format(len(set(train_all_combined_sentence_tokens))))


Train sentence1 dataset size is 100000
Train sentence2 dataset size is 100000
Val sentence1 dataset size is 1000
Val sentence2 dataset size is 1000

Total number of tokens in sentence1 train dataset is 1294135
Total number of tokens in sentence2 train dataset is 743372
Total number of tokens in combined sent1 & sent2 train dataset is 2037507

Total number of *unique* tokens in sentence1 train dataset is 14131
Total number of *unique* tokens in sentence2 train dataset is 15225
Total number of *unique* tokens in sent1 & sent2 train dataset is 19642


In [9]:
#build vocabularies for sentence1 and sentence2
from collections import Counter

max_vocab_size = 10000
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1

def build_vocab(all_tokens):
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(max_vocab_size))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token

token2id_combined_sent, id2token_combined_sent = build_vocab(train_all_combined_sentence_tokens)


In [10]:
# Lets check the dictionary by loading random token from it
import random 

random_token_id = random.randint(0, len(id2token_combined_sent)-1)
random_token = id2token_combined_sent[random_token_id]

print ("Token id {} ; token {}".format(random_token_id, id2token_combined_sent[random_token_id]))
print ("Token {}; token id {}".format(random_token, token2id_combined_sent[random_token]))

Token id 7819 ; token fame
Token fame; token id 7819


In [11]:
# convert token to id in the dataset.  After running this cell we will have converted the word tokens to indices
def token2index_dataset(tokens_data,token2id):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data

train_sentence1_data_indices = token2index_dataset(train_sentence1_tokens,token2id_combined_sent)
train_sentence2_data_indices = token2index_dataset(train_sentence2_tokens,token2id_combined_sent)
val_sentence1_data_indices = token2index_dataset(val_sentence1_tokens,token2id_combined_sent)
val_sentence2_data_indices = token2index_dataset(val_sentence2_tokens,token2id_combined_sent)

# double checking
print ("Train sentence1 dataset size is {}".format(len(train_sentence1_data_indices)))
print ("Train sentence2 dataset size is {}".format(len(train_sentence2_data_indices)))
print ("Val sentence1 dataset size is {}".format(len(val_sentence1_data_indices)))
print ("Val sentence2 dataset size is {}".format(len(val_sentence2_data_indices)))

Train sentence1 dataset size is 100000
Train sentence2 dataset size is 100000
Val sentence1 dataset size is 1000
Val sentence2 dataset size is 1000


In [12]:
#visualize a random sentence1 and sentence2 paired training example
rand_training_example = random.randint(0, len(train_sentence1) - 1)
print (train_sentence1_tokens[rand_training_example])
print(train_sentence1_data_indices[rand_training_example])

print (train_sentence2_tokens[rand_training_example])
print(train_sentence2_data_indices[rand_training_example])

['a', 'green', 'sign', 'lists', 'the', 'directions', 'names', 'and', 'distance', 'of', 'two', 'different', 'towns']
[2, 57, 214, 9375, 3, 1391, 6593, 8, 559, 10, 13, 584, 6594]
['the', 'sign', 'is', 'blue']
[3, 214, 5, 31]


In [13]:
#Check average, max, min sentence lengths to determine word padding
total_sent1_len=0
total_sent2_len=0
sent1_lens=[]
sent2_lens=[]
for i in range(0,len(train_sentence1_tokens)):
    total_sent1_len+=len(train_sentence1_tokens[i])
    total_sent2_len+=len(train_sentence2_tokens[i])
    sent1_lens.append(len(train_sentence1_tokens[i]))
    sent2_lens.append(len(train_sentence2_tokens[i]))

avg1=total_sent1_len/len(train_sentence1)
avg2=total_sent2_len/len(train_sentence2)
print("sentence1 average is: "+str(avg1)+", std dev is: "+str(np.std(sent1_lens))+", max is: "+str(max(sent1_lens))+", min is: "+str(min(sent1_lens)))
print("sentence2 average is: "+str(avg2)+", std dev is: "+str(np.std(sent2_lens))+", max is: "+str(max(sent2_lens))+", min is: "+str(min(sent2_lens)))


sentence1 average is: 12.94135, std dev is: 5.755700667816214, max is: 78, min is: 2
sentence2 average is: 7.43372, std dev is: 3.0907033118046123, max is: 38, min is: 1


In [39]:
import numpy as np
import torch
from torch.utils.data import Dataset

class NewsGroupDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """
    
    def __init__(self, data_list1, data_list2, target_list, MAX_SENTENCE_LENGTH):
        """
        @param data_list: list of newsgroup tokens 
        @param target_list: list of newsgroup targets 

        """
        self.data_list1 = data_list1
        self.data_list2 = data_list2
        self.MAX_SENTENCE_LENGTH=MAX_SENTENCE_LENGTH
        self.target_list = target_list
        assert (len(self.data_list1) == len(self.target_list) == len(self.data_list2))

    def __len__(self):
        return len(self.data_list1)
        
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        token_idx1 = self.data_list1[key][:self.MAX_SENTENCE_LENGTH]
        token_idx2 = self.data_list2[key][:self.MAX_SENTENCE_LENGTH]
        label = self.target_list[key]
        return [token_idx1, token_idx2, len(token_idx1), len(token_idx2), label]
    

In [40]:
train_dataset = NewsGroupDataset(train_sentence1_data_indices,train_sentence2_data_indices, train_targets, 25)
val_dataset = NewsGroupDataset(val_sentence1_data_indices, val_sentence2_data_indices, val_targets, 25)

In [41]:
print("sample lengths:")
for i in range(10):
    print(train_dataset[i][2])
    print(train_dataset[i][3])

sample lengths:
16
9
21
7
9
9
14
9
9
9
12
5
9
8
6
9
16
4
25
5


In [42]:
## example output

print("sent1: x {};\ny {}".format(train_dataset[0][0], train_dataset[0][2]))
print("sent2: x {};\ny {}".format(train_dataset[0][0], train_dataset[0][2]))

sent1: x [2, 21, 23, 4, 2, 101, 20, 30, 7, 2, 720, 1934, 2, 351, 10, 50];
y 16
sent2: x [2, 21, 23, 4, 2, 101, 20, 30, 7, 2, 720, 1934, 2, 351, 10, 50];
y 16


In [43]:
MAX_SENTENCE_LENGTH=25

def newsgroup_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    data_list1 = []
    data_list2 = []
    label_list = []
    length_list1 = []
    length_list2 = []
    #print("collate batch: ", batch[0][0])
    #batch[0][0] = batch[0][0][:MAX_SENTENCE_LENGTH]
    for datum in batch:
        label_list.append(datum[4])
        length_list1.append(datum[2])
        length_list2.append(datum[3])
    # padding
    for datum in batch:
        padded_vec1 = np.pad(np.array(datum[0]), 
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[2])), 
                                mode="constant", constant_values=0)
        data_list1.append(padded_vec1)
        
    for datum in batch:
        
        padded_vec2 = np.pad(np.array(datum[1]), 
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[3])), 
                                mode="constant", constant_values=0)
        data_list2.append(padded_vec2)
        
    return [torch.from_numpy(np.array(data_list1)), torch.from_numpy(np.array(data_list2)), torch.LongTensor(length_list1), torch.LongTensor(length_list2), torch.LongTensor(label_list)]



In [44]:
BATCH_SIZE = 32

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)

In [45]:
#sent1 example
for i,(data1, data2, lengths1, lengths2, labels) in enumerate(train_loader):
    print(data1)
    print(data1.shape)
    print(lengths1)
    print(lengths1.shape)
    print(labels)
    print(labels.shape)
    break

tensor([[  13,   22,   93, 3020,   16,    2,  619,   71,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0],
        [  56,   14,   24, 5965,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0],
        [1082,   14,   30,   82,    2,   28,   94,   24,    2, 1658,  106,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0],
        [  49,    9,   56,   14,    4,    2,  246,    8,   13,   22,   40,   47,
          159,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0],
        [  18, 4486, 4487,    5, 2139, 1022,   33,  428,  324,   37,    3, 2094,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0],
        [   2,  171, 1131,  706,  172,    4, 3360,   36,    2, 1611,    0,    0,
            0,    0,    0,   

### Bag-of-Words model in PyTorch

Next, we will implement a Bag of Words in PyTorch -- as an `nn.Module`.

A `nn.Module` can really be any function, but it is often used to implement layers, functions and models. Note that you can also nest modules.

Importantly, modules need to have their `forward()` method overridden, and very often you will want to override the `__init__` method as well. 

The `__init__` method sets up the module. This is also often where the internal modules and parameters are initialized.

The `forward` method defines what happens when you *apply* the module.

In the background, PyTorch makes use of your code in the forward method and determines how to implement back-propagation with it - but all you need to do is to define the forward pass!

In [53]:
test1 = [[1,2,3,4],[5,6,7,8],[9,10,11,12]]
test2 = [[1,2,3,],[5,6,7],[9,10,11]]

def concat_matrices(M1,M2):
    new_matrix = []
    for i in range(len(M1)):
        new_matrix.append(M1[i] + M2[i])
        
    return new_matrix

concat_matrices(test1, test2)

test3 = [1,2,3,4,5]
test4 = [5,5,5,5,5]
list( map(add, test3, test4) )

[6, 7, 8, 9, 10]

In [137]:
# First import torch related libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

class BagOfWords(nn.Module):
    """
    BagOfWords classification model
    """
    def __init__(self, vocab_size, emb_dim):
        """
        @param vocab_size: size of the vocabulary. 
        @param emb_dim: size of the word embedding
        """
        super(BagOfWords, self).__init__()
        # pay attention to padding_idx 
        self.embed = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.linear = nn.Linear(emb_dim,3)
    
    def forward(self, data1, data2, lengths1, lengths2):
        """
        
        @param data: matrix of size (batch_size, max_sentence_length). Each row in data represents a 
            review that is represented using n-gram index. Note that they are padded to have same length.
        @param length: an int tensor of size (batch_size), which represents the non-trivial (excludes padding)
            length of each sentences in the data.
        """
        
        data = torch.cat((data1,data2), dim=1, out=None)
        out = self.embed(data)
        length = lengths1.add(lengths2)
        
        out = torch.sum(out, dim=1)
        out /= length.view(length.size()[0],1).expand_as(out).float()
     
        # return logits
        out = self.linear(out.float())
        return out

In [138]:
emb_dim = 100
model = BagOfWords(len(id2token_combined_sent), emb_dim)

In [139]:
model.linear

Linear(in_features=100, out_features=3, bias=True)

In [140]:
model.embed

Embedding(10002, 100, padding_idx=0)

In [141]:
model.embed.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.5976, -0.5552,  0.2011,  ..., -0.7586,  1.8274,  1.7724],
        [ 1.3177, -0.0551, -0.9264,  ...,  0.5579,  1.7113,  0.3485],
        ...,
        [ 0.7141,  0.4026,  0.7386,  ...,  1.0490, -0.1133,  0.7727],
        [ 0.6313, -0.2025,  0.6294,  ...,  0.4419,  0.7739, -0.1178],
        [-1.0753, -0.3663,  0.1696,  ...,  0.5914, -0.7005,  0.7240]],
       requires_grad=True)

In [142]:
model.embed.weight.shape

torch.Size([10002, 100])

### Loss Function and Optimizer

Note that in our Bag of Words model we haven't applied softmax to the output of linear layer. Why?
We use `nn.CrossEntropyLoss()` to train. From pytorch documentation for `nn.CrossEntropyLoss()` ( https://pytorch.org/docs/stable/nn.html ) - this criterion combines `nn.LogSoftmax()` and `nn.NLLLoss()` in one single class. So, this is actually exactly the same as minimizing the log likelihood after applying softmax.

In [143]:
# Criterion and Optimizer
criterion = torch.nn.CrossEntropyLoss()  

learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [144]:
for x in model.parameters():
    print(x.shape)

torch.Size([10002, 100])
torch.Size([3, 100])
torch.Size([3])


### Training the Model

In [145]:
#this will run but not really work because we have not combined sent1, sent2 representations

num_epochs = 10 # number epoch to train

# Function for testing the model
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for data1, data2, lengths1, lengths2, labels in loader:
        data_batch1, data_batch2, length_batch1, length_batch2, label_batch = data1, data2, lengths1, lengths2, labels
        outputs = F.softmax(model(data_batch1, data_batch2, length_batch1, length_batch2), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]
        
        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)

for epoch in range(num_epochs):
    for i, (data1, data2, lengths1, lengths2, labels) in enumerate(train_loader):
        model.train()
        data_batch1, data_batch2, length_batch1, length_batch2, label_batch = data1, data2, lengths1, lengths2, labels
        optimizer.zero_grad()
        outputs = model(data_batch1, data_batch2, length_batch1, length_batch2)
        loss = criterion(outputs, label_batch)
        loss.backward()
        optimizer.step()
        # validate every 100 iterations
        if i > 0 and i % 100 == 0:
            # validate
            val_acc = test_model(val_loader, model)
            print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                epoch+1, num_epochs, i+1, len(train_loader), val_acc))


Epoch: [1/10], Step: [101/3125], Validation Acc: 42.4
Epoch: [1/10], Step: [201/3125], Validation Acc: 44.3
Epoch: [1/10], Step: [301/3125], Validation Acc: 47.7
Epoch: [1/10], Step: [401/3125], Validation Acc: 45.6
Epoch: [1/10], Step: [501/3125], Validation Acc: 49.3
Epoch: [1/10], Step: [601/3125], Validation Acc: 47.9
Epoch: [1/10], Step: [701/3125], Validation Acc: 49.0
Epoch: [1/10], Step: [801/3125], Validation Acc: 52.2
Epoch: [1/10], Step: [901/3125], Validation Acc: 49.1
Epoch: [1/10], Step: [1001/3125], Validation Acc: 48.8
Epoch: [1/10], Step: [1101/3125], Validation Acc: 48.8
Epoch: [1/10], Step: [1201/3125], Validation Acc: 51.0
Epoch: [1/10], Step: [1301/3125], Validation Acc: 51.2
Epoch: [1/10], Step: [1401/3125], Validation Acc: 50.6
Epoch: [1/10], Step: [1501/3125], Validation Acc: 49.3
Epoch: [1/10], Step: [1601/3125], Validation Acc: 50.3
Epoch: [1/10], Step: [1701/3125], Validation Acc: 51.3
Epoch: [1/10], Step: [1801/3125], Validation Acc: 50.7
Epoch: [1/10], Step

In [1]:
def inverse_tokenize(sent):
    output = ''
    for word in sent:
        output += id2token_combined_sent[word]
        output += ' '
    return output

def six_classified_sentences(model_save_path):
    model_type, combine_type, vocab_size, emb_dim, l2, reg = model_save_path.split('_')
    model=BagOfWords(int(vocab_size), int(emb_dim), combine_type, model_type)
    model.load_state_dict(torch.load(model_save_path))
    model.eval()
    
    misclassifieds = []
    classifieds = []
    
    #index = -1
    while len(misclassifieds) < 3 or len(classifieds) < 3:
        #index += 1
        index = np.random.randint(len(val_dataset))
        sent1, sent2, len1, len2, label = val_dataset[index]
        output = F.softmax(model(torch.tensor([sent1]), torch.tensor([sent2]), torch.tensor([len1]), torch.tensor([len2])))
        predicted = output.max(1, keepdim=True)[1]
        if int(predicted) != label and len(misclassifieds) < 3:
            misclassifieds.append([inverse_tokenize(sent1), inverse_tokenize(sent2), label, int(predicted), index])
        elif int(predicted) == label and len(classifieds) < 3:
            classifieds.append([inverse_tokenize(sent1), inverse_tokenize(sent2), label, int(predicted), index])
    return misclassifieds, classifieds

incorrect, correct = six_classified_sentences('LG_product_15002_200_l2_reg')

print('The following three instances were classified incorrectly:')
print('#########################################################')
for instance in incorrect:
    sent1, sent2, true_label, pred_label, index = instance
    print('sentence 1: ', sent1)
    print('sentence 2: ', sent2)
    print('Real Label: ', true_label)
    print('Predicted Label: ', pred_label)
    print('')

print('')
print('')
print('The following three instances were classified correctly:')
print('#########################################################')
for instance in correct:
    sent1, sent2, true_label, pred_label, index = instance
    print('sentence 1: ', sent1)
    print('sentence 2: ', sent2)
    print('Real Label: ', true_label)
    print('Predicted Label: ', pred_label)
    print('')

NameError: name 'BagOfWords' is not defined

### Section 3.2

In [ ]:
mnli_train_df = pd.read_csv('mnli_train.tsv', sep="\t")
mnli_val_df = pd.read_csv('mnli_val.tsv',sep="\t")

#Creates a list of genres represented in data
all_genres = []
for genre in mnli_train_df['genre']:
    if not genre in all_genres:
        all_genres.append(genre)
        
def tokenize_with_vocab(dataset,vocab):
    output = []
    for sentence in dataset:
        tokens = tokenize(sentence)
        index_list = [vocab[token] if token in vocab.keys() else 1 for token in tokens]
        output.append(index_list)
        
    return output

for genre in all_genres:
    if not os.path.exists('val_sentence2_' +genere+'.p'):
        dataset = mnli_val_df.loc[mnli_val_df['genre'] == genre]
        sentence1_dataset = list(dataset['sentence1'])
        sentence2_dataset = list(dataset['sentence2'])
        val_sentence1 = tokenize_with_vocab(sentence1_dataset, token2id_combined_sent)
        val_sentence2 = tokenize_with_vocab(sentence2_dataset, token2id_combined_sent)
        pkl.dump(val_sentence1, open("val_sentence1_" + genre +".p", "wb"))
        pkl.dump(val_sentence2, open("val_sentence2_" + genre +".p", "wb"))
            
val_sentence1_telephone = pkl.load(open("val_sentence1_telephone.p", "rb"))
val_sentence2_telephone = pkl.load(open("val_sentence2_telephone.p", "rb"))
val_sentence1_fiction = pkl.load(open("val_sentence1_fiction.p", "rb"))
val_sentence2_fiction = pkl.load(open("val_sentence2_fiction.p", "rb"))
val_sentence1_slate = pkl.load(open("val_sentence1_slate.p", "rb"))
val_sentence2_slate = pkl.load(open("val_sentence2_slate.p", "rb"))
val_sentence1_government = pkl.load(open("val_sentence1_government.p", "rb"))
val_sentence2_government = pkl.load(open("val_sentence2_government.p", "rb"))
val_sentence1_travel = pkl.load(open("val_sentence1_travel.p", "rb"))
val_sentence2_travel = pkl.load(open("val_sentence2_travel.p", "rb"))

telephone_label = list(le.transform(mnli_val_df.loc[mnli_val_df['genre'] == 'telephone']['label']))
fiction_label = list(le.transform(mnli_val_df.loc[mnli_val_df['genre'] == 'fiction']['label']))
slate_label = list(le.transform(mnli_val_df.loc[mnli_val_df['genre'] == 'slate']['label']))
government_label = list(le.transform(mnli_val_df.loc[mnli_val_df['genre'] == 'government']['label']))
travel_label = list(le.transform(mnli_val_df.loc[mnli_val_df['genre'] == 'travel']['label']))



val_dataset_telephone = NewsGroupDataset(val_sentence1_telephone,val_sentence2_telephone, telephone_label, 35)
val_dataset_fiction = NewsGroupDataset(val_sentence1_fiction,val_sentence2_fiction, fiction_label, 35)
val_dataset_slate = NewsGroupDataset(val_sentence1_slate,val_sentence2_slate, slate_label, 35)
val_dataset_government = NewsGroupDataset(val_sentence1_government,val_sentence2_government, government_label, 35)
val_dataset_travel = NewsGroupDataset(val_sentence1_travel,val_sentence2_travel, travel_label, 35)

telephone_loader = torch.utils.data.DataLoader(dataset=val_dataset_telephone, 
                                        batch_size=len(telephone_label),
                                        collate_fn=newsgroup_collate_func,
                                        shuffle=True)

fiction_loader = torch.utils.data.DataLoader(dataset=val_dataset_fiction, 
                                        batch_size=len(fiction_label),
                                        collate_fn=newsgroup_collate_func,
                                        shuffle=True)

slate_loader = torch.utils.data.DataLoader(dataset=val_dataset_slate, 
                                        batch_size=len(slate_label),
                                        collate_fn=newsgroup_collate_func,
                                        shuffle=True)

government_loader = torch.utils.data.DataLoader(dataset=val_dataset_government, 
                                        batch_size=len(government_label),
                                        collate_fn=newsgroup_collate_func,
                                        shuffle=True)

travel_loader = torch.utils.data.DataLoader(dataset=val_dataset_travel, 
                                        batch_size=len(travel_label),
                                        collate_fn=newsgroup_collate_func,
                                        shuffle=True)

In [ ]:
def section_3point2(model_save_path):
    model_type, combine_type, vocab_size, emb_dim, l2, reg = model_save_path.split('_')
    model=BagOfWords(int(vocab_size), int(emb_dim), combine_type, model_type)
    model.load_state_dict(torch.load(model_save_path))
    
    telephone_accuracy = test_model(telephone_loader, model)[0]
    fiction_accuracy = test_model(fiction_loader, model)[0]
    slate_accuracy = test_model(slate_loader, model)[0]
    government_accuracy = test_model(government_loader, model)[0]
    travel_accuracy = test_model(travel_loader, model)[0]
    
    return pd.DataFrame({'telephone accuracy':[telephone_accuracy], 'fiction accuracy': [fiction_accuracy], 
            'slate accuracy': [slate_accuracy], 'government': [government_accuracy], 'travel accuracy':[travel_accuracy]})

best_LG_model_path = 'LG_product_15002_200_l2_reg'
best_NN_model_path = 'LG_product_15002_200_l2_reg'

LG_genre_df = pd.DataFrame(section_3point2(best_LG_model_path))
NN_genre_df = pd.DataFrame(section_3point2(best_NN_model_path))

table_3point2 = pd.concat([LG_genre_df, NN_genre_df])
table_3point2['Model Type'] = ['LG', 'NN']
table_3point2 = table_3point2[table_3point2.columns[-1:].tolist() + table_3point2.columns[:-1].tolist()]
table_3point2